In [ ]:
#import libraries

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import model_selection
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



In [ ]:
#load the datasets

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
#get an idea of the data

train.head(5)

In [ ]:
test.head(5)

In [ ]:
#descriptive statistics

train.describe()

In [ ]:
#drop columns we don't need

train.drop(['PassengerId','Cabin', 'Ticket', 'Name'], axis=1, inplace = True)
test.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace = True)

In [ ]:
#create a list with both datasets

datasets = [train, test]

In [ ]:
#Replace NaN with values (clean dataset)

for dataset in datasets:    
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#convert to categorical

label = LabelEncoder()
for dataset in datasets:    
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])


In [ ]:
train.head(5)

In [ ]:
#set features and target/label
features = ['Sex','Pclass', 'Embarked','SibSp', 'Parch', 'Age', 'Fare']
target = ['Survived']

features_for_test = ['PassengerId', 'Sex','Pclass', 'Embarked','SibSp', 'Parch', 'Age', 'Fare']


In [ ]:
#create dummy columns from categorical

train_dummy = pd.get_dummies(train[features])
test_dummy = pd.get_dummies(test[features_for_test])


train_dummy.head()

In [ ]:
#Correlations

for x in features:
    if train[x].dtype != 'float64':
        print('Survival Correlation by:', x)
        print(train[[x, target[0]]].groupby(x, as_index=False).mean())
        print('-'*10, '\n')

In [ ]:
#correlation heatmap
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(train)

In [ ]:
#split the dataset
new_features = train_dummy.columns.tolist()
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_dummy[new_features], train[target], train_size=0.8)

In [ ]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train_x[new_features], train_y)


In [ ]:
predictions = clf.predict(test_x)

In [ ]:
print("Train Accuracy :: ", accuracy_score(train_y, clf.predict(train_x)))
print("Test Accuracy  :: ", accuracy_score(test_y, predictions))

In [ ]:
test_dummy['Survived'] = clf.predict(test_dummy[new_features])

In [ ]:
test_dummy[['PassengerId','Survived']].to_csv('predictions.csv',index=False)